In [1]:
!cd

E:\Projets\Dog and cat


In [1]:
!kaggle competitions download -c dogs-vs-cats -p Data/

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [1]:
import os
import shutil
import zipfile
import matplotlib.image as image
from io import BytesIO
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.layers import Conv2D,MaxPooling2D,Activation,Dense,Flatten,Dropout
from keras.models import Sequential
from keras.initializers import RandomNormal,RandomUniform
from keras.optimizers import Adam,RMSprop
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

C:\Users\nites\.conda\envs\deeplearning\lib\site-packages\ipykernel\parentpoller.py:113: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  warnings.warn("""Parent poll failed.  If the frontend dies,


In [2]:
print('Extracting files ....')
with zipfile.ZipFile('Data/dogs-vs-cats.zip') as zfiles:
    for file in zfiles.namelist():
        try:
            zfiledata = BytesIO(zfiles.read(file))
            efile = zipfile.ZipFile(zfiledata)
            efile.extractall('data/')
        except:
            print('An error occured')
print('Extraction completed ....')
print('Extracted file location (data/).... ')

Extracting files ....
An error occured
Extraction completed ....
Extracted file location (data/).... 


In [3]:
train_data_path = './Data/train/'
test_data_path = './Data/test1/'

In [94]:
def get_data_ready(path):
    print('Loading Files .... ')
    print('Please Wait ....')
    label, file_list ,=[],[]
    #data = []
    for files in os.walk(path):
        for file in files[2]:
            filename, extension = os.path.splitext(file)
            if extension == '.jpg':
                #load_path = path+file
                #img = image.imread(load_path)
                #data.append(img)
                if len(filename.split('.'))>1:
                    file_list.append(file)
                    label.append(file.split('.')[0])
                else:
                    file_list.append(file)
            else:
                print('Not a .jpg file  the actual extension is {}'.format(extension))
                
#    data = np.asarray(data)
    label = np.asarray(label)
    print('Files Loading Complete ....')
    return label,file_list

In [95]:
train_labels,train_filename = get_data_ready(train_data_path)

Loading Files .... 
Please Wait ....
Files Loading Complete ....


In [96]:
test_labels,test_filename = get_data_ready(test_data_path)

Loading Files .... 
Please Wait ....
Files Loading Complete ....


In [97]:
train_df = pd.DataFrame(data=zip(train_filename,train_labels),columns=['filename','labels'],)
test_df = pd.DataFrame(data=test_filename,columns=['filename'])

In [98]:
train_data,valid_data = train_test_split(train_df,
                                         random_state=42,                            
                                         test_size=0.25,
                                        shuffle=True)

In [99]:
data_gen =ImageDataGenerator(zoom_range=0.2,
                             rescale=(1./255.),
                             horizontal_flip=True,
                             shear_range=0.2,
                             validation_split=0.25)

In [100]:
valid_data.reset_index(drop=True,inplace=True)
train_data.reset_index(drop=True,inplace=True)

In [101]:
train_data_generator = data_gen.flow_from_dataframe(dataframe = train_df,
                                                    directory = './Data/train/',
                                                    x_col = 'filename',
                                                    y_col = 'labels',
                                                    target_size = (128,128),
                                                    subset = 'training',
                                                    batch_size = 32,
                                                    class_mode = 'binary')

valid_data_generator = data_gen.flow_from_dataframe(dataframe = train_df,
                                                    directory = 'Data/train/',
                                                    x_col = 'filename',
                                                    y_col = 'labels',
                                                    target_size = (128,128),
                                                    subset = 'validation',
                                                    batch_size = 32,
                                                    class_mode = 'binary')

Found 18750 validated image filenames belonging to 2 classes.
Found 6250 validated image filenames belonging to 2 classes.


In [102]:
train_data['filename'].nunique()

18750

In [103]:
model1 = Sequential()

model1.add(Conv2D(filters = 32 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu',
                  input_shape = (128,128,3)))

model1.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))

model1.add(Dropout(0.25))

model1.add(Conv2D(filters = 64 ,
                  kernel_size = (3,3),
                  strides = (1,1),
                  padding = 'valid',
                  activation='relu'))

model1.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1), 
                        padding='valid'))

model1.add(Conv2D(filters = 64 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))

model1.add(Dropout(0.25))
model1.add(Conv2D(filters = 64 ,
                  kernel_size = (3,3),
                  strides = (1,1),
                  padding = 'valid',
                  activation='relu'))

model1.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))


model1.add(Conv2D(filters = 128 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu'))

model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(units=64,activation = 'relu'))
model1.add(Dense(units=128,activation = 'relu'))
model1.add(Dense(units=256,activation = 'relu'))
model1.add(Dense(units=1,activation = 'sigmoid'))


model1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 30, 30, 64)        36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 29, 29, 64)       

In [104]:
opt = RMSprop(learning_rate= 0.1)
model1.compile(optimizer=opt,
               loss = 'binary_crossentropy',
               metrics = 'accuracy')

In [105]:
model1_summary = model1.fit(train_data_generator,
                                      shuffle=True,
                                      epochs = 10,
                                      validation_data=valid_data_generator)

Epoch 1/10
586/586 [==============================] - 692s 1s/step - loss: 2949678336.0000 - accuracy: 0.6085 - val_loss: 0.8063 - val_accuracy: 0.0000e+00
Epoch 2/10
586/586 [==============================] - 355s 606ms/step - loss: 0.6395 - accuracy: 0.6663 - val_loss: 1.1977 - val_accuracy: 0.0000e+00
Epoch 3/10
586/586 [==============================] - 352s 601ms/step - loss: 0.6390 - accuracy: 0.6667 - val_loss: 1.0697 - val_accuracy: 0.0000e+00
Epoch 4/10
586/586 [==============================] - 353s 602ms/step - loss: 0.6384 - accuracy: 0.6667 - val_loss: 1.1195 - val_accuracy: 0.0000e+00
Epoch 5/10
586/586 [==============================] - 352s 600ms/step - loss: 0.6384 - accuracy: 0.6667 - val_loss: 1.2693 - val_accuracy: 0.0000e+00
Epoch 6/10
586/586 [==============================] - 353s 602ms/step - loss: 0.6393 - accuracy: 0.6667 - val_loss: 1.0871 - val_accuracy: 0.0000e+00
Epoch 7/10
586/586 [==============================] - 351s 600ms/step - loss: 0.6383 - accurac

In [106]:
model2 = Sequential()

model2.add(Conv2D(filters = 32 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu',
                  input_shape = (128,128,3)))

model2.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))

model2.add(Dropout(0.25))

model2.add(Conv2D(filters = 64 ,
                  kernel_size = (3,3),
                  strides = (1,1),
                  padding = 'valid',
                  activation='relu'))

model2.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1), 
                        padding='valid'))

model2.add(Conv2D(filters = 128 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))

model2.add(Dropout(0.25))
model2.add(Conv2D(filters = 256 ,
                  kernel_size = (3,3),
                  strides = (1,1),
                  padding = 'valid',
                  activation='relu'))

model2.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))


model2.add(Conv2D(filters = 256 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu'))

model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(units=64,activation = 'relu'))
model2.add(Dense(units=128,activation = 'relu'))
model2.add(Dense(units=256,activation = 'relu'))
model2.add(Dense(units=1,activation = 'sigmoid'))


model2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 30, 30, 128)       73856     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 29, 29, 128)      

In [107]:
opt = Adam(learning_rate= 0.1)
model2.compile(optimizer=opt,
               loss = 'binary_crossentropy',
               metrics = 'accuracy')

In [ ]:
model2_summary = model2.fit(train_data_generator,
                            shuffle=True,
                            epochs = 10,
                            validation_data=valid_data_generator)

Epoch 1/10
586/586 [==============================] - 877s 1s/step - loss: 201962128.0000 - accuracy: 0.6556 - val_loss: 1.1177 - val_accuracy: 0.0000e+00
Epoch 2/10
404/586 [===================>..........] - ETA: 3:41 - loss: 0.6435 - accuracy: 0.6704

In [ ]:
test_gen = ImageDataGenerator(rescale=(1./255.))
test_data_generator = test_gen.flow_from_dataframe(test_df,
                                                   directory='./Data/test1/',
                                                   x_col='filename',
                                                   y_col=None,
                                                   class_mode=None,
                                                   target_size=(128,128),
                                                   batch_size=32)

In [ ]:
pred1 = model1.predict(test_data_generator)

In [ ]:
pred2 = model2.predict(test_data_generator)